In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기

- Human Labeling된 데이터인 SKT , 현대백화점, 현대모비스, S-Oil, 기업은행, 셀트리온 데이터 셋을 이용하여 각 정확도를 올린다.

- 순서
    1. 데이터 불러오기
    2. 데이터 분리
    3. Title 의 토큰화 및 불용어 제거
    4. Label 의 원핫인코딩
    5. 길이가 다른 title의 정형화
    6. LSTM을 통한 딥러닝
    7. 정확도 확인

In [2]:
df_skt = pd.read_csv('../handmade_sentitest/trainingset/SKT_traindata_482.csv')
df_train_skt = df_skt.iloc[:200]
df_test_skt = df_skt.iloc[200:]

In [79]:
df_test_skt

,id,Title,label
200,0,"정부, 구글 지도 해외 반출 불허 구글이 보완책 수용 안해 (상보)",0
201,1,"미래부, 2차 유료방송 발전방안 공개토론회 개최",0
202,2,"SKT, 전용폰 구입 고객에 T1 롤드컵 우승 기념품 증정",1
203,3,'갤S7'사면 페이백 50만원 준다더니제2의 거성 모바일 사태?,0
204,4,케이블TV 가입자도 내년 1월부터 SKT 요금할인받는다,1
...,...,...,...
476,78,"스웨덴 에릭슨""4차 산업혁명 협력할 韓 스타트업 찾습니다 ,0\n4187,SKT '...",1
477,79,"CJ헬로비전 작년 영업이익 59.4% 감소한 429억원…""M&A 등 불확실성 해소,...",1
478,80,SKT 2019년까지 11조원 투자,1
479,81,통신사 커버리지맵 정확도 SKT 1위 '오차 0%',1


In [3]:
df_mobis = pd.read_csv('../handmade_sentitest/trainingset/현대모비스_traindata_200.csv', encoding = 'CP949')
# df_mobis

In [4]:
df_hb = pd.read_csv('../handmade_sentitest/trainingset/현대백화점_traindata_200.csv')
# df_hb

In [5]:
df_S_Oil = pd.read_csv('../handmade_sentitest/trainingset/S-Oil_traindata_200.csv')
df_S_Oil = df_S_Oil.head(200)
# df_S_Oil.head(200)

In [6]:
df_IBK = pd.read_csv('../handmade_sentitest/trainingset/기업은행_traindata_200.csv')
# df_IBK

In [7]:
df_celt = pd.read_csv('../handmade_sentitest/trainingset/셀트리온_traindata_200.csv')
df_celt=df_celt.head(200)
#df_celt

In [8]:
df_KAL = pd.read_csv('../handmade_sentitest/trainingset/대한항공_testdata_300.csv', header=None)
df_celt=df_celt.head(200)
# df_KAL=df_KAL.head(300)
df_KAL=df_KAL.head(299)
df_KAL.columns = ['id','Title','label']
# df_KAL

In [9]:
df = pd.concat([df_train_skt, df_mobis, df_hb, df_S_Oil,df_IBK,df_celt])

In [10]:
# df = df[['id','Title','label']]
# df

,id,Title,label
0,0.0,국민안전로봇 2021년까지 개발…'로봇물고기' 전철 피할까?,0
1,1.0,'국정원 해킹 의혹' 상임위 앞두고 與野 전초전,0
2,2.0,인터넷티비(TV)결합상품 인터넷가입 비교사이트 통해 통신사별 장단점 비교해야 유리,0
3,3.0,SKT LG KT 인터넷가입 비교사이트 ‘펭귄통신’ 설치 당일 현금지원 이벤트,0
4,4.0,설 연휴 고향 가는 길 언제 출발해야 덜 막힐까,0
...,...,...,...
195,1172.0,"원 이퀴티 파트너스 포, 엘.피., 셀트리온헬스케어 주식 631만주 장내매도",-1
196,1630.0,"코스닥, 장 초반 하락세…640선 또 '위태'",-1
197,696.0,“ 3000% ”터질 바이오 단돈 3천원대! 2019년 마지막 新대장 바이오 온다!,1
198,1563.0,"코스닥, 개인 '사자'에 700 향해 ↑",1


In [10]:
from sklearn.model_selection import train_test_split

## 데이터셋 구성
- Train Dataset : skt 200개, 모비스 200개, 현백 200개, 에쓰오일 200개, 기업은행 200개, 셀트리온 200개
- Test Dataset : skt 282개

In [70]:
# Train 데이터와 Test 데이터 분리
X_train = df['Title']
y_train = df['label']
X_test = df_test_skt['Title']
y_test = df_test_skt['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

## khaiii모델 적용 Sentence Tokenizing

In [15]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

In [74]:
X_train[:3]

0                국민안전로봇 2021년까지 개발…'로봇물고기' 전철 피할까?
1                       '국정원 해킹 의혹' 상임위 앞두고 與野 전초전
2    인터넷티비(TV)결합상품 인터넷가입 비교사이트 통해 통신사별 장단점 비교해야 유리
Name: Title, dtype: object

In [91]:
# khaiii로 기사 제목 tokenizing
stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', 
              '하다', '까지','…','?','.',',','!','"',"'",'로','고','(',')','[',']','ㄹ까']

X_train_khaiii = []
for sentence in X_train:
    morphs = []
    for word in api.analyze(sentence):
        for morph in word.morphs:
            if morph.lex not in stop_words:
                morphs.append(morph.lex)
    X_train_khaiii.append(morphs)

        
X_test_khaiii = []
for sentence in X_test:
    morphs = []
    for word in api.analyze(sentence):
        for morph in word.morphs:
            if morph.lex not in stop_words:
                morphs.append(morph.lex)
    X_test_khaiii.append(morphs)

In [90]:
X_train_khaiii

[['국민', '안전로봇', '2021', '년', '개발', '봇물고기', '전철', '피하', 'ㄹ까'],
 ['국정원', '해킹', '의혹', '상임위', '앞두', '與野', '전초전'],
 ['인터넷티비',
  'TV',
  '결합',
  '상품',
  '인터넷가입',
  '비교사이',
  '트',
  '통하',
  '여',
  '통신사',
  '별',
  '장단점',
  '비교',
  '하',
  '여야',
  '유리']]

In [88]:
X_test_khaiii

[['정부',
  ',',
  '구글',
  '지도',
  '해외',
  '반출',
  '불허',
  '구글',
  '이',
  '보완책',
  '수용',
  '안',
  '하',
  '여',
  '(',
  '상',
  '보',
  ')'],
 ['미래부', ',', '2', '차', '유료', '방송', '발전', '방안', '공개', '토론회', '개최'],
 ['SKT', ',', '전용폰', '구입', '고객', '에', 'T', '1', '롤드컵', '우승', '기념품', '증정'],
 ["'",
  '갤',
  'S',
  '7',
  "'",
  '사면',
  '페이백',
  '50',
  '만',
  '원',
  '주',
  'ㄴ다',
  '더니',
  '제',
  '2',
  '의',
  '거성',
  '모바일',
  '사태',
  '?'],
 ['케이블', 'TV', '가입자', '도', '내년', '1', '월', '부터', 'SKT', '요금', '할인', '받', '는다'],
 ['"', '가성비폰', '전성', '시대', '중가폰', '시장서', '선방', '"'],
 ['美',
  'AT',
  '&',
  'T',
  ',',
  '타',
  '임',
  '워',
  '너',
  '인수',
  '국내',
  '통신',
  '·',
  '미디어',
  '社',
  '도',
  'M',
  '&',
  'A',
  '힘받',
  '는다'],
 ['신경',
  '민',
  '의원',
  '이통사',
  ',',
  '유심',
  '(',
  'USIM',
  ')',
  '독점',
  '판매',
  '못',
  '하',
  'ㄴ다'],
 ['갤노트', '7', '가입자', '50', '만', '명', '교환', '시작', '번호', '이동도', '자유'],
 ['SKT',
  ',',
  '2030',
  '대상',
  "'",
  '밴드',
  'YT',
  '요금',
  '제',
  "'",
  '출시',
  '대학내',
  '·'

In [ ]:
#불용어 제거

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

X_train_khaiii2 = [] 
for w in X_train_khaiii: 
    if w not in stop_words: 
        X_train_khaiii2.append(w)


In [17]:
# 단어만 추출해서 리스트 생성
X_train_khaiii2 = []
for i in range(len(X_train_khaiii)):
    word = X_train_khaiii[i]
    temp_list = []
    for j in range(len(word)):
        temp_list.append(X_train_khaiii[i][j][0])
    X_train_khaiii2.append(temp_list)
        
X_test_khaiii2 = []
for i in range(len(X_test_khaiii)):
    word = X_test_khaiii[i]
    temp_list = []
    for j in range(len(word)):
        temp_list.append(X_test_khaiii[i][j][0])
    X_test_khaiii2.append(temp_list)

In [76]:
X_train_khaiii2

[['국민',
  '안전로봇',
  '2021',
  '년',
  '까지',
  '개발',
  '…',
  "'",
  '로',
  '봇물고기',
  "'",
  '전철',
  '피하',
  'ㄹ까',
  '?'],
 ["'", '국정원', '해킹', '의혹', "'", '상임위', '앞두', '고', '與野', '전초전'],
 ['인터넷티비',
  '(',
  'TV',
  ')',
  '결합',
  '상품',
  '인터넷가입',
  '비교사이',
  '트',
  '통하',
  '여',
  '통신사',
  '별',
  '장단점',
  '비교',
  '하',
  '여야',
  '유리'],
 ['SKT',
  'LG',
  'KT',
  '인터넷가입',
  '비교사이트',
  '‘',
  '펭귄통신',
  '’',
  '설치',
  '당일',
  '현금',
  '지원',
  '이벤트'],
 ['설', '연휴', '고향', '가', '는', '길', '언제', '출발', '하', '여야', '덜', '막히', 'ㄹ까'],
 ['인터넷가입',
  'KT',
  '·',
  'SK',
  '·',
  'LGU',
  '플',
  '러',
  '스',
  '인터넷',
  'TV',
  '결합',
  '설치',
  '시',
  '현금사은품',
  '설치',
  '당일',
  '지급',
  '소비자',
  '몰리',
  'ㄴ다'],
 ["'",
  '운서',
  'SK',
  '뷰',
  '스',
  '카이시',
  '티',
  "'",
  '견본',
  '주택',
  ',',
  '개관',
  '3',
  '일',
  '간',
  '1',
  '만',
  '여',
  '명',
  '방문'],
 ['카카오', '·', '삼성화', '재', '디지털', '보험사', '도전'],
 ['[',
  '2019',
  '여성',
  '리더스포',
  '럼',
  ']',
  '"',
  '지속가능',
  '하',
  'ㄴ',
  '일',
  '위하',
  '여',
  '회복탄력',

In [18]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

### 단어길이 추출

In [78]:
li_train = []
for i in range(len(X_train_khaiii)):
   li_train.append(len(X_train_khaiii[i]))
print('훈련데이터 최대 길이 :',max(li_train))

li_test = []
for i in range(len(X_test_khaiii)):
   li_test.append(len(X_test_khaiii[i]))
print('테스트데이터 최대 길이 :',max(li_test))

훈련데이터 최대 길이 : 31
테스트데이터 최대 길이 : 73


In [19]:
li_train = []
for i in range(len(X_train_khaiii2)):
   li_train.append(len(X_train_khaiii2[i]))
print('훈련데이터 최대 길이 :',max(li_train))

li_test = []
for i in range(len(X_test_khaiii2)):
   li_test.append(len(X_test_khaiii2[i]))
print('테스트데이터 최대 길이 :',max(li_test))

훈련데이터 최대 길이 : 31
테스트데이터 최대 길이 : 73


### max_words를 35000에서 100으로 줄임(test 길이 중 가장 큰것은 73)

In [79]:
from keras.preprocessing.text import Tokenizer
max_words = 200
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_khaiii) 
X_train_token = tokenizer.texts_to_sequences(X_train_khaiii) 
X_test_token = tokenizer.texts_to_sequences(X_test_khaiii)

AttributeError: 'tuple' object has no attribute 'lower'

In [46]:
from keras.preprocessing.text import Tokenizer
max_words = 500
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_khaiii2) 
X_train_token = tokenizer.texts_to_sequences(X_train_khaiii2) 
X_test_token = tokenizer.texts_to_sequences(X_test_khaiii2)

### One-Hot encoding 2가지 방법 - 2번째 방법 사용 (to_categorize)

In [19]:
# 1. 
import numpy as np
y_train_encode = []
y_test_encode = []
for i in range(len(y_train)):
    if y_train.iloc[i] == 1:
        y_train_encode.append([0, 0, 1]) 
    elif y_train.iloc[i] == 0:
        y_train_encode.append([0, 1, 0]) 
    elif y_train.iloc[i] == -1:
        y_train_encode.append([1, 0, 0]) 
        
for i in range(len(y_test)):
    if y_test.iloc[i] == 1:
        y_test_encode.append([0, 0, 1]) 
    elif y_test.iloc[i] == 0:
        y_test_encode.append([0, 1, 0])
    elif y_test.iloc[i] == -1:
        y_test_encode.append([1, 0, 0]) 
        
y_train_encode=np.array(y_train_encode)
y_test_encode=np.array(y_test_encode)

In [20]:
len(y_train_encode)

1022

In [21]:
# 2. 
# to_categorical 방법은 0 1 -1 순서로 인코딩된다!
from keras.utils import to_categorical

y_train_cata = to_categorical(y_train, num_classes = 3)
y_test_cata = to_categorical(y_test, num_classes = 3)

In [22]:
y_train_cata

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

### max_len 20 to 100으로 변경 ; 단어 길이 고려

In [43]:
max_len = 100 # pad_sequesces이용, 전체 데이터의 길이를 100로 맞춘다
X_train = pad_sequences(X_train_token, maxlen=max_len)
X_test = pad_sequences(X_test_token, maxlen=max_len)

### LSTM 모델

optimizer = adam, epoch=10일 때, 테스트 결과 54.80%의 정확도를 보인다. 

### earlystopping과 modelcheckpoint 추가
### embedding 64로 변경

In [47]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128,activation='tanh'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_cata, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)

Epoch 1/20


InvalidArgumentError:  indices[6,92] = 523 is not in [0, 500)
	 [[node sequential_8/embedding_8/embedding_lookup (defined at <ipython-input-47-37656d51514a>:10) ]] [Op:__inference_train_function_32783]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_8/embedding_8/embedding_lookup:
 sequential_8/embedding_8/embedding_lookup/31578 (defined at /usr/lib/python3.6/contextlib.py:81)

Function call stack:
train_function


In [45]:
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 1.9089 - accuracy: 0.4982

 테스트 정확도 : 49.82%


In [93]:
#optimizer = adam
model2 = Sequential()
model2.add(Embedding(max_words, 64))
model2.add(LSTM(128))
model2.add(Dense(3, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model2.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

Epoch 1/10
109/110 [============================>.] - ETA: 0s - loss: 1.0051 - accuracy: 0.5229

110/110 [==============================] - 6s 58ms/step - loss: 1.0039 - accuracy: 0.5250 - val_loss: 0.9920 - val_accuracy: 0.5610
Epoch 2/10
110/110 [==============================] - ETA: 0s - loss: 0.8902 - accuracy: 0.5787

110/110 [==============================] - 6s 56ms/step - loss: 0.8902 - accuracy: 0.5787 - val_loss: 0.8665 - val_accuracy: 0.6748
Epoch 3/10
110/110 [==============================] - ETA: 0s - loss: 0.8028 - accuracy: 0.6260

110/110 [==============================] - 6s 56ms/step - loss: 0.8028 - accuracy: 0.6260 - val_loss: 0.8766 - val_accuracy: 0.5854
Epoch 4/10
110/110 [==============================] - ETA: 0s - loss: 0.7257 - accuracy: 0.6624

110/110 [==============================] - 6s 53ms/step - loss: 0.7257 - accuracy: 0.6624 - val_loss: 0.7431 - val_accuracy: 0.6911
Epoch 5/10
109/110 [============================>.] - ETA: 0s - loss: 0.6908 - accuracy: 0.6743

110/110 [==============================] - 6s 52ms/step - loss: 0.6909 - accuracy: 0.6752 - val_loss: 0.8022 - val_accuracy: 0.6260
Epoch 6/10
109/110 [============================>.] - ETA: 0s - loss: 0.6569 - accuracy: 0.6844

110/110 [==============================] - 6s 54ms/step - loss: 0.6540 - accuracy: 0.6870 - val_loss: 0.9340 - val_accuracy: 0.5528
Epoch 7/10
110/110 [==============================] - ETA: 0s - loss: 0.6320 - accuracy: 0.6997

110/110 [==============================] - 6s 53ms/step - loss: 0.6320 - accuracy: 0.6997 - val_loss: 0.9198 - val_accuracy: 0.5854
Epoch 8/10
109/110 [============================>.] - ETA: 0s - loss: 0.6007 - accuracy: 0.7183

110/110 [==============================] - 6s 53ms/step - loss: 0.6019 - accuracy: 0.7179 - val_loss: 0.9831 - val_accuracy: 0.5854
Epoch 00008: early stopping


In [94]:
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 1.1205 - accuracy: 0.5907

 테스트 정확도 : 59.07%


In [95]:
#model3 ; activation = relu
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model3 = Sequential()
model3.add(Embedding(max_words, 64))
model3.add(LSTM(128))
model3.add(Dense(3, activation='relu'))
model3.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model3.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

print("\n 테스트 정확도 : {:.2f}%".format(model3.evaluate(X_test,y_test_cata)[1]*100))

Epoch 1/10
110/110 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2957

110/110 [==============================] - 6s 55ms/step - loss: nan - accuracy: 0.2957 - val_loss: nan - val_accuracy: 0.1220
Epoch 2/10
110/110 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2912

110/110 [==============================] - 5s 47ms/step - loss: nan - accuracy: 0.2912 - val_loss: nan - val_accuracy: 0.1220
Epoch 3/10
110/110 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2912

110/110 [==============================] - 5s 47ms/step - loss: nan - accuracy: 0.2912 - val_loss: nan - val_accuracy: 0.1220
Epoch 4/10
110/110 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2912

110/110 [==============================] - 5s 47ms/step - loss: nan - accuracy: 0.2912 - val_loss: nan - val_accuracy: 0.1220
Epoch 00004: early stopping
9/9 [==============================] - 0s 17ms/step - loss: nan - accuracy: 0.5623

 테스트 정확도 : 56.23%


In [96]:
#model5 ; activation = softmax optimzier = nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model5 = Sequential()
model5.add(Embedding(max_words, 64))
model5.add(LSTM(128))
model5.add(Dense(3, activation='softmax'))
model5.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model5.fit(X_train, y_train_cata, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)

print("\n 테스트 정확도 : {:.2f}%".format(model5.evaluate(X_test,y_test_cata)[1]*100))

Epoch 1/20
110/110 [==============================] - ETA: 0s - loss: 1.0217 - accuracy: 0.5287

110/110 [==============================] - 6s 56ms/step - loss: 1.0217 - accuracy: 0.5287 - val_loss: 1.0397 - val_accuracy: 0.5366
Epoch 2/20
110/110 [==============================] - ETA: 0s - loss: 0.9218 - accuracy: 0.5505

110/110 [==============================] - 6s 53ms/step - loss: 0.9218 - accuracy: 0.5505 - val_loss: 0.8670 - val_accuracy: 0.6260
Epoch 3/20
110/110 [==============================] - ETA: 0s - loss: 0.8115 - accuracy: 0.6297

110/110 [==============================] - 6s 55ms/step - loss: 0.8115 - accuracy: 0.6297 - val_loss: 0.8760 - val_accuracy: 0.6260
Epoch 4/20
110/110 [==============================] - ETA: 0s - loss: 0.7860 - accuracy: 0.6588

110/110 [==============================] - 6s 55ms/step - loss: 0.7860 - accuracy: 0.6588 - val_loss: 0.8029 - val_accuracy: 0.6748
Epoch 5/20
110/110 [==============================] - ETA: 0s - loss: 0.7104 - accuracy: 0.6797

...

110/110 [==============================] - 6s 56ms/step - loss: 0.6750 - accuracy: 0.6924 - val_loss: 0.7971 - val_accuracy: 0.6667
Epoch 7/20
109/110 [============================>.] - ETA: 0s - loss: 0.6567 - accuracy: 0.6972

110/110 [==============================] - 6s 55ms/step - loss: 0.6584 - accuracy: 0.6943 - val_loss: 0.8570 - val_accuracy: 0.6585
Epoch 8/20
110/110 [==============================] - ETA: 0s - loss: 0.6318 - accuracy: 0.7125

110/110 [==============================] - 6s 55ms/step - loss: 0.6318 - accuracy: 0.7125 - val_loss: 0.8842 - val_accuracy: 0.6341
Epoch 9/20
110/110 [==============================] - ETA: 0s - loss: 0.6091 - accuracy: 0.7216

110/110 [==============================] - 6s 55ms/step - loss: 0.6091 - accuracy: 0.7216 - val_loss: 0.9117 - val_accuracy: 0.5854
Epoch 10/20
110/110 [==============================] - ETA: 0s - loss: 0.5847 - accuracy: 0.7225

110/110 [==============================] - 6s 54ms/step - loss: 0.5847 - accuracy: 0.7225 - val_loss: 1.1374 - val_accuracy: 0.5610
Epoch 00010: early stopping
9/9 [==============================] - 0s 16ms/step - loss: 1.1799 - accuracy: 0.5552

 테스트 정확도 : 55.52%


In [23]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense
from keras.layers import Flatten

In [24]:
model6 = Sequential()
model6.add(Embedding(max_words, 64))
model6.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model6.add(MaxPooling1D(pool_size=2))
model6.add(Flatten())
model6.add(Dense(3, activation='relu'))
model6.add(Dense(3, activation='sigmoid'))

model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model6.fit(X_train, y_train_cata, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)

print("\n 테스트 정확도 : {:.2f}%".format(model6.evaluate(X_test,y_test_cata)[1]*100))

ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.

In [32]:
model6.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.